In [ ]:
import pandas as pd
fname_sales_train = '../input/competitive-data-science-predict-future-sales/sales_train.csv'
df_sales_train = pd.read_csv(fname_sales_train)
df_sales_train

In [ ]:
df_sales_train.isnull().sum()

In [ ]:
fname_shops = '../input/competitive-data-science-predict-future-sales/shops.csv'
df_shops = pd.read_csv(fname_shops)
df_shops

In [ ]:
df_shops.isnull().sum()

In [ ]:
fname_items='../input/competitive-data-science-predict-future-sales/items.csv'
df_items=pd.read_csv(fname_items)
df_items

In [ ]:
df_items.isnull().sum()

In [ ]:
fname_item_categories='../input/competitive-data-science-predict-future-sales/item_categories.csv'
df_item_categories=pd.read_csv(fname_item_categories)
df_item_categories

In [ ]:
df_item_categories.isnull().sum()

In [ ]:
fname_test='../input/competitive-data-science-predict-future-sales/test.csv'
df_test=pd.read_csv(fname_test)
df_test

**Pre-processing**

Merge tables

In [ ]:
df_master=pd.merge(df_sales_train,df_shops,how='outer',on='shop_id')
df_master

In [ ]:
df_master=pd.merge(df_master,df_items,how='outer',on='item_id')
df_master

In [ ]:
df_master=pd.merge(df_master,df_item_categories,how='outer',on='item_category_id')
df_master

**Convert date to datetime**

(o dia vem primeiro no dataset então "dayfirst"=true)

In [ ]:
df_master['date']=pd.to_datetime(df_master['date'],dayfirst=True)
df_master

**Merge days**

In [ ]:
df_master['date'] = df_master['date'].apply(lambda x: x.strftime('%Y-%m') if x is not pd.NaT else pd.NaT)
df_master

Quantidade de items vendidos por mês

In [ ]:
df = df_master[['date', 'shop_id', 'item_id', 'item_cnt_day']]
df = df.groupby(['date', 'shop_id', 'item_id']).sum()
df

In [ ]:
import numpy as np

df = df.pivot_table(index=['shop_id','item_id'], columns='date', values='item_cnt_day', fill_value=0, aggfunc=np.sum)
df

In [ ]:
df = df.reset_index()
df.columns.name = None
df

**Merge past data to test one**

In [ ]:
df_test_src=pd.merge(df_test,df,how='left', on=['shop_id','item_id'])
df_test_src=df_test_src.drop(['ID'],axis=1).fillna(0)
df_test_src

**Get explanatory and objective variables**

In [ ]:
X_train = df.drop(['2015-10'], axis=1)
X_train

In [ ]:
y_train = df['2015-10']
y_train

In [ ]:
X_test = df_test_src.drop(['2013-01'], axis=1)
X_test

**LSTM Modeling**

Reshape data

In [ ]:
X_train_nn = X_train.values[:, 2:]
X_train_nn.shape

In [ ]:
y_train_nn = y_train.values
y_train_nn.shape

In [ ]:
X_test_nn = X_test.values[:, 2:]
X_test_nn.shape

**Normalization**

In [ ]:
from sklearn.preprocessing import RobustScaler
X_scaler = RobustScaler()

X_train_nn = X_scaler.fit_transform(X_train_nn)
X_test_nn = X_scaler.transform(X_test_nn)

y_scaler = RobustScaler()
y_train_nn=y_scaler.fit_transform(np.expand_dims(y_train_nn,axis=-1)).squeeze()


**Neural Network**

In [ ]:
import tensorflow as tf

input = tf.keras.layers.Input(shape=(33,1))

x = input
x = tf.keras.layers.GRU(32, return_sequences=True, dropout=0.1)(x)
x = tf.keras.layers.GRU(32, return_sequences=True, dropout=0.1)(x)
x = tf.keras.layers.GRU(32, dropout=0.1)(x)

output = tf.keras.layers.Dense(1)(x)

model = tf.keras.models.Model(input,output)
model.summary()

In [ ]:
model.compile(optimizer='adam', loss=tf.keras.losses.MeanSquaredError(),metrics=['accuracy'])

**Learning**

In [ ]:
epochs = 100
batch_size = 256
MSE=[]
accus=[]
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)]

history = model.fit(np.expand_dims(X_train_nn, axis=-1),
                    y_train_nn,
                    validation_split=0.1,
                    epochs=epochs,
                    batch_size=batch_size,
                    callbacks=callbacks)
MSE.append(history.history['loss'])
accus.append(history.history['accuracy'])

In [ ]:
MSE

In [ ]:
MSE=[17.23740577697754,
  17.146400451660156,
  17.06780242919922,
  17.00035285949707,
  16.937820434570312,
  16.876121520996094,
  16.806711196899414,
  16.72635269165039,
  16.653059005737305,
  16.641311645507812,
  16.586013793945312,
  16.544328689575195,
  16.477075576782227,
  16.44742774963379,
  16.36529541015625,
  16.329736709594727,
  16.272869110107422,
  16.232526779174805,
  16.382827758789062,
  16.121000289916992,
  16.110342025756836,
  16.075496673583984,
  16.019672393798828,
  16.0389461517334,
  15.980767250061035]

In [ ]:
import numpy as np
[np.mean(MSE)]

In [ ]:
MSE2=[16.560233157614004,
 16.20846902756464,
 16.192753728230795,
 15.83425122499466,
 16.408649103982107,
 16.110182637753695,
 16.353180817195348,
 15.871306971499795,
 16.476158752441407,
 16.14610550620339,
 16.042625253850762,
 16.147459250230057,
 16.39347590169599,
 16.431691106160482,
 16.0304728558189,
 16.29587086764249,
 16.50674347082774,
 16.141746811244797,
 16.04782921075821,
 15.95643138885498,
 16.403288394212723,
 16.175094561143354,
 16.139255046844482,
 16.160969560796563,
 16.22540261195256,
 16.16890223821004,
 16.426226395827072,
 16.164062573359562,
 16.436739955629623,
 15.996581569794685,
 16.006873267037527,
 16.032642110188803,
 16.444861037390574,
 16.436265182495116,
 16.139064967632294,
 15.92740535736084,
 16.362799326578777,
 16.039993180169,
 16.174950880162857,
 15.991812578837077,
 16.535628235858418,
 16.043943157902472,
 16.08322678671943,
 15.858921156989204,
 16.33932500897032,
 16.06690712632804,
 16.07255354794589,
 16.092534482479095,
 16.45778072797335,
 16.133479754130047,
 16.22783317565918,
 16.263992743058637,
 16.488136609395344,
 15.967597007751465,
 16.152947545051575,
 15.893114240545975,
 16.340637058019638,
 16.523025671641033,
 16.467392444610596,
 15.73178495679583,
 16.516398906707764,
 16.2178731398149,
 16.137426326149388,
 15.958148454364977]

In [ ]:
#model.compile(loss)

**Prediction**

In [ ]:
y_pred = model.predict(np.expand_dims(X_test_nn, -1))
y_pred

In [ ]:
y_pred = y_scaler.inverse_transform(y_pred).squeeze()
y_pred

In [ ]:
answer = pd.concat([df_test['ID'], pd.Series(y_pred.squeeze(), name='item_cnt_month')], axis=1)
answer

In [ ]:
submission='./submission.csv'
answer.to_csv(submission, index=False)